In [ ]:
large_sets = 

In [ ]:
pc2 = unstressed_protein_coding_pca.loadings.ix['pc_2'].sort_values(ascending=False)
large_sets = mm10GO.GO[(mm10GO.GO['nGenes'] > 25) & (mm10GO.GO['nGenes'] < 500)] 

In [ ]:
result = {}
in_set = {}
for name, genes in large_sets['Ensembl Gene ID'].iteritems():
    result[name] = pc2.copy()
    in_set[name] = {gene_id: True for gene_id in genes}

result = pd.DataFrame(result)
in_set = pd.DataFrame(in_set)
in_set = in_set.fillna(False)

incomplete_in_set = in_set.T

for gene in result.index.difference(in_set.index):
    incomplete_in_set[gene] = False
    
in_set = incomplete_in_set[result.index].T

#is_set = in_set.reindex(result.index, fill_value=False)

hit_values = result.copy()
miss_values = result.copy()

In [ ]:
hit_values[~in_set] = 0
hit_values = np.abs(hit_values)
miss_values[:] = 0
miss_values[~in_set] = 1

In [ ]:
def enrichment_score_vectorized(hit_values, miss_values):
    normalized_hit_values = hit_values / hit_values.sum()
    normalized_miss_values = miss_values / miss_values.sum()

    enrichment_score = normalized_hit_values.cumsum() - normalized_miss_values.cumsum()
    return enrichment_score

def get_largest_score_vectorized(enrichment_score):
    combined_scores = pd.concat({"max_score": enrichment_score.max(), "min_score": enrichment_score.min()}).unstack()
    return combined_scores.apply(lambda x: x.max_score if np.abs(x.max_score) > np.abs(x.min_score) else x.min_score, axis=0)

In [ ]:
enrichment_score = enrichment_score_vectorized(hit_values, miss_values)

In [ ]:
largest_enrichment = get_largest_score_vectorized(enrichment_score)

In [ ]:
results = {}
for x in range(100):
    index = list(hit_values.index)
    #np.random.shuffle(df.index.get_values())
    random.shuffle(index)
    shuffled_hit_values = hit_values.copy().ix[index]
    shuffled_miss_values = miss_values.copy().ix[index]

    shuffled_hit_values.reset_index()
    shuffled_miss_values.reset_index()
    results[x] = get_largest_score_vectorized(enrichment_score_vectorized(shuffled_hit_values, shuffled_miss_values))
shuffled_results = pd.concat(results).unstack()
